In [10]:
from face2voice.models.SpeakerEncoder import SpeakerEncoder
from torchvision.transforms import ToTensor
from openvoice.api import ToneColorConverter
import torch
from openvoice import se_extractor
import numpy as np

In [11]:
tone_color_converter = ToneColorConverter(f'checkpoints/tone_conv/config.json', device="cpu")
tone_color_converter.load_ckpt(f'checkpoints/tone_conv/checkpoint.pth')

/Users/mcmk/projects/face2voice/.venv/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Loaded checkpoint 'checkpoints/tone_conv/checkpoint.pth'
missing/unexpected keys: [] []


In [12]:
speaker_encoder = SpeakerEncoder(config_path="checkpoints/tone_conv/config.json", ckpt_path="checkpoints/tone_conv/checkpoint.pth")

Loaded checkpoint 'checkpoints/tone_conv/checkpoint.pth'
missing/unexpected keys: [] []


In [13]:
with open("resources/A.J._Buckley_1zcIwhmdeo4_0000001.npz") as spec_file:
    spec = np.load("resources/A.J._Buckley_1zcIwhmdeo4_0000001.npz")["spec"]
spec = torch.from_numpy(spec).to(torch.float32)

In [14]:
tgt_emb = speaker_encoder.encode_single(audio="resources/1zcIwhmdeo4_0000001_audio.wav", input="audio")

OpenVoice version: v2
[(0.0, 10.6735)]
after vad: dur = 10.673
Audio encode data:  <class 'torch.Tensor'> torch.Size([1, 513, 919]) tensor(0.6398)
torch.Size([1, 1, 256])


In [15]:
src_emb = speaker_encoder.encode_single(audio="resources/example_reference.mp3", input="audio")

OpenVoice version: v2
[(0.0, 58.8188125)]
after vad: dur = 58.81798185941043
Audio encode data:  <class 'torch.Tensor'> torch.Size([1, 513, 844]) tensor(0.3639)
Audio encode data:  <class 'torch.Tensor'> torch.Size([1, 513, 844]) tensor(0.3796)
Audio encode data:  <class 'torch.Tensor'> torch.Size([1, 513, 844]) tensor(0.3636)
Audio encode data:  <class 'torch.Tensor'> torch.Size([1, 513, 844]) tensor(0.3526)
Audio encode data:  <class 'torch.Tensor'> torch.Size([1, 513, 844]) tensor(0.3360)
Audio encode data:  <class 'torch.Tensor'> torch.Size([1, 513, 844]) tensor(0.3509)
torch.Size([1, 1, 256])


In [16]:
tgt_emb = torch.tensor(tgt_emb).transpose(1, 2)
src_emb = torch.tensor(src_emb).transpose(1, 2)
tgt_emb.shape, src_emb.shape

/var/folders/nw/7l6j9hqx7_3gyljpmz012p400000gn/T/ipykernel_47461/3801821416.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tgt_emb = torch.tensor(tgt_emb).transpose(1, 2)
/var/folders/nw/7l6j9hqx7_3gyljpmz012p400000gn/T/ipykernel_47461/3801821416.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  src_emb = torch.tensor(src_emb).transpose(1, 2)


(torch.Size([1, 256, 1]), torch.Size([1, 256, 1]))

In [17]:
speaker_encoder.tone_color_converter.convert(audio_src_path="resources/example_reference.mp3", src_se=src_emb, tgt_se=tgt_emb, output_path="outputs/npztest.wav")